Transcripts: https://survivor.fandom.com/wiki/The_Marooning/Transcript  
Scores: https://www.truedorktimes.com/survivor/boxscores/s5.htm  

In [1]:
import pandas as pd
import numpy as np
import re

import requests
from bs4 import BeautifulSoup

## Test for S1E1

In [2]:
url = 'https://www.truedorktimes.com/survivor/boxscores/s1.htm'

page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')

In [3]:
doc_id = re.search(r'(?<=https://docs.google.com/spreadsheets/d/).*(?=/edit?)',
                   str(soup)).group()

sheet = 'e1'

doc_link = f'https://docs.google.com/spreadsheets/d/{doc_id}/gviz/tq?tqx=out:csv&sheet={sheet}'

In [4]:
data = pd.read_csv(doc_link)

In [5]:
df1 = (data
 .iloc[0:int(len(data)/2 - 1), 0:16]
 .drop(['Unnamed: 4','Unnamed: 10','Unnamed: 12','Unnamed: 13','VFT','JVF'],
       axis = 1)
 .rename(columns = {'Unnamed: 0':'player_name',
                    'ChW':'challenge_wins',
                    'ChA':'challenge_appearances',
                    'SO':'sit_outs',
                    'VFB':'votes_for_bootee',
                    'VAP':'votes_against_player',
                    'TotV':'total_votes',
                    'TCA':'tribal_council_appearances',
                    'tot days':'total_days',
                    'exile days':'exile_days'
                   })
)

In [6]:
df2 = (data
 .iloc[int(len(data)/2):int(len(data)-1), 0:7]
 .reset_index(drop = True)
 .rename(columns = {'Unnamed: 0':'player_name',
                    'ChW':'reward_chl_win',
                    'ChA':'reward_chl_teammates',
                    'SO':'reward_chl_win_perc',
                    'Unnamed: 4':'immun_chl_win',
                    'VFB':'immun_chl_teammates',
                    'VAP':'immun_chl_win_perc'
                    # 'TotV':'total_win_perc'
                   })
)

In [7]:
s1e1 = (df1
        .merge(df2, how = 'left', on = 'player_name')
        .assign(season = 1,
                episode = 1)
       )

## Loop for All Seasons / Episodes

In [25]:
episodes = range(1,12)
seasons = range(1,41)

In [26]:
df_dict = {}

for szn in seasons:
    url = str('https://www.truedorktimes.com/survivor/boxscores/s' +
              str(szn) +'.htm')
    
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    doc_id = re.search(r'(?<=https://docs.google.com/spreadsheets/d/).*(?=(/edit?|/">))',
                   str(soup)).group()
    
    for epi in episodes:
        sheet = str('e'+str(1))
        doc_link = f'https://docs.google.com/spreadsheets/d/{doc_id}/gviz/tq?tqx=out:csv&sheet={sheet}'
        
        df_name = str('s'+str(szn)+'e'+str(epi))
        df_dict[df_name] = pd.read_csv(doc_link)
        
    print('Season',szn,'complete.')

Season 1 complete.
Season 2 complete.


In [27]:
for szn in seasons:
    
    for epi in episodes:
        
        df_name = str('s'+str(szn)+'e'+str(epi))
        data = df_dict[df_name]
        
        df1 = (data
               .iloc[0:int(len(data)/2 - 1), 0:16]
               .drop(['Unnamed: 4','Unnamed: 10','Unnamed: 12','Unnamed: 13','VFT','JVF'],
                     axis = 1)
               .rename(columns = {'Unnamed: 0':'player_name',
                    'ChW':'challenge_wins',
                    'ChA':'challenge_appearances',
                    'SO':'sit_outs',
                    'VFB':'votes_for_bootee',
                    'VAP':'votes_against_player',
                    'TotV':'total_votes',
                    'TCA':'tribal_council_appearances',
                    'tot days':'total_days',
                    'exile days':'exile_days'
                   })
              )
        
        df2 = (data
               .iloc[int(len(data)/2):int(len(data)-1), 0:7]
               .reset_index(drop = True)
               .rename(columns = {'Unnamed: 0':'player_name',
                    'ChW':'reward_chl_win',
                    'ChA':'reward_chl_teammates',
                    'SO':'reward_chl_win_perc',
                    'Unnamed: 4':'immun_chl_win',
                    'VFB':'immun_chl_teammates',
                    'VAP':'immun_chl_win_perc'
                    # 'TotV':'total_win_perc'
                   })
              )
        
        combo = (df1
                 .merge(df2, how = 'left', on = 'player_name')
                 .assign(season = szn,
                         episode = epi)
                )
        
        df_name = str('s'+str(szn)+'e'+str(epi))
        df_dict[df_name] = combo
        
    print('Season',szn,'complete.')

Season 1 complete.
Season 2 complete.


In [28]:
full_df = pd.DataFrame(columns = df_dict['s1e1'].columns)

for i in df_dict:
    full_df = pd.concat([full_df,df_dict[i]],
                        ignore_index = True)

In [29]:
full_df

,player_name,challenge_wins,challenge_appearances,sit_outs,votes_for_bootee,votes_against_player,total_votes,tribal_council_appearances,total_days,exile_days,reward_chl_win,reward_chl_teammates,reward_chl_win_perc,immun_chl_win,immun_chl_teammates,immun_chl_win_perc,season,episode
0,Richard,0.000,0.125,NaN,0.0,NaN,8.0,1.0,3.0,NaN,0.0,8.0,0.000,NaN,NaN,NaN,1,1
1,Kelly W,0.000,0.125,NaN,0.0,NaN,8.0,1.0,3.0,NaN,0.0,8.0,0.000,NaN,NaN,NaN,1,1
2,Rudy,0.000,0.125,NaN,1.0,3.0,8.0,1.0,3.0,NaN,0.0,8.0,0.000,NaN,NaN,NaN,1,1
3,Sue,0.000,0.125,NaN,1.0,NaN,8.0,1.0,3.0,NaN,0.0,8.0,0.000,NaN,NaN,NaN,1,1
4,Sean,0.000,0.125,NaN,1.0,NaN,8.0,1.0,3.0,NaN,0.0,8.0,0.000,NaN,NaN,NaN,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336,Skupin,0.000,0.125,NaN,1.0,NaN,8.0,1.0,3.0,NaN,0.0,8.0,0.000,NaN,NaN,NaN,2,11
337,Kimmi,0.000,0.125,NaN,1.0,NaN,8.0,1.0,3.0,NaN,0.0,8.0,0.000,NaN,NaN,NaN,2,11
338,Mitchell,0.125,0.125,NaN,NaN,NaN,NaN,NaN,3.0,NaN,1.0,8.0,0.125,NaN,NaN,NaN,2,11
339,Maralyn,0.125,0.125,NaN,NaN,NaN,NaN,NaN,3.0,NaN,1.0,8.0,0.125,NaN,NaN,NaN,2,11
